In [38]:
#from PyPDF2 import PdfReader
import PyPDF2
import pandas as pd
import openai
import os
from IPython.display import display, HTML

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')
state = "Colorado"
psco_tar = "PSCo_Electric_Entire_Tariff.pdf"

In [47]:
prompt = f"""
Please extract detailed electric tariff rates from the provided PDF document and categorize 
them line by line as follows:

<Residential Rates>:
- List all residential tariff rates with rate names and associated costs.
- Include any information about seasonal variations in residential rates.
- Mention any time-of-use (TOU) details specific to residential rates.
- Identify any incentives or programs related to residential rates.

<Commercial Rates>:
- List all commercial tariff rates with rate names and associated costs.
- Include any information about seasonal variations in commercial rates.
- Mention any time-of-use (TOU) details specific to commercial rates.
- Identify any incentives or programs related to commercial rates.

Please provide this information based on the content of the PDF document. Thank you.
```{text}```
"""

# Define the get_completion function
def get_completion(prompt, model="gpt-4-1106-preview"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # This is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

# Define a function to process a chunk of pages
def process_chunk(chunk):
    chunk_text = ""
    for page in chunk:
        text = page.extract_text()
        chunk_text += text
    # Call the get_completion function with the chunk text
    completion = get_completion(chunk_text)
    return completion

# Open the PDF file in read-binary mode
with open(psco_tar, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    total_pages = 10
    chunk_size = 5  # Set the number of pages per chunk
    response_string = ""

    for start_page in range(0, total_pages, chunk_size):
        end_page = min(start_page + chunk_size, total_pages)

        # Create a list to store pages in the current chunk
        chunk = []
        for page_num in range(start_page, end_page):
            page = pdf_reader.pages[page_num]
            chunk.append(page)

        # Call the processing function for the current chunk and append the result
        completion = process_chunk(chunk)
        response_string += completion

# Close the PDF file
pdf_file.close()

# The final result is stored in the response_string variable
display(HTML(response_string))
#print(response_string)